In [ ]:
### 只做单模型，其他的内容，等等再说，现在把单模型的表现提升上去再说,这份code的目标只是为了提升模型的分类效率

In [1]:
import sys
sys.path.append("../")
import pandas as pd
import numpy as np
from dateutil.parser import parse
from sklearn import metrics
import datetime
import collections
import time
import random

import model_ml as mm
import feat_engineering as fe
import feat_selection as fs
import model_tunning as mt
from param_config import config

Using TensorFlow backend.
C:\Users\Leo Mao\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
dfTrain = pd.read_csv(config.original_train_data_path)
dfPred = pd.read_csv(config.original_pred_data_path)
predictors = dfPred.columns.tolist()[4:]
check_missing = ['PartI_1','PartII_1','PartIII_1','PartIV_1','PartV_1','PartVI_1']

###清理异常Y值
#异常大值
dfTrain = dfTrain.loc[dfTrain['Y']<dfTrain['Y'].max()]

#性别空
dfTrain = dfTrain.loc[~dfTrain['sex'].isnull()]

dfTrain = dfTrain.reset_index(drop=True)

saveY = dfTrain['Y'].tolist()

dfTrain.loc[dfTrain['Y']>dfTrain['Y'].quantile(0.9),'Y'] = 1
dfTrain.loc[dfTrain['Y']!=1,'Y'] = 0

In [3]:
def FeatAll(train,pred,rankMinus=False,missing=False,missing_step2=False):
    dfAll = pd.concat([train,pred])
    dfAll = dfAll.reset_index(drop=True)
    dfAll['date'] = (pd.to_datetime(dfAll['date']) - parse('2017-10-09')).dt.days
    
    ###RF填补缺失值
    if missing:
        dfAll = fe.fillna_RF(dfAll,predictors,True)

    
    ###尝试处理异常值
    '''for var in predictors:
        dfAll.loc[dfAll[var]>=dfAll[var].quantile(0.99)*3,var] = dfAll[var].quantile(0.99)*3'''
        #dfAll.loc[dfAll[var]>=df[var].quantile(0.99)*1.5,var] = np.nan
    ###根据年龄性别去做分类，鉴别不同人的百分比
    dfAll['ageBin'] = 0
    ageBin = [20,30,40,50,60]
    for i in range(len(ageBin)):
        dfAll.loc[dfAll['age']>ageBin[i],'ageBin'] = i+1
    dfAll['sexAge'] = dfAll[['sex','ageBin']].apply(lambda x: x[1]*2+x[0])
    dfAllPcentBySexAge = fe.pcent_by_other_col(dfAll,{'sex':predictors,'ageBin':predictors,'sexAge':predictors},['ID'])
    del dfAll['age']
    del dfAll['sexAge']
    if rankMinus:
        try:
            for i in ['_sex_','_ageBin_','_sexAge']:
                dfrankMinusTmp = pd.read_csv('../../Cache/rankMinus%s.csv'%i[:-1])
                if i == '_sex_':
                    dfrankMinus = dfrankMinusTmp.copy()
                else:
                    dfrankMinus = dfrankMinus.merge(dfrankMinusTmp,'inner','ID')
        except:
            for i in ['_sex_','_ageBin_','_sexAge']:
                varList = [var for var in dfAllPcentBySexAge.columns if i in var]
                dfrankMinusTmp = fe.var_minus(dfAllPcentBySexAge,varList,['ID'],0)
                dfrankMinusTmp.to_csv('../../Cache/rankMinus%s.csv'%i[:-1],index=False)
                if i == '_sex_':
                    dfrankMinus = dfrankMinusTmp.copy()
                else:
                    dfrankMinus = dfrankMinus.merge(dfrankMinusTmp,'inner','ID')  

    
    ###目前不想使用日期，觉得用处不大
    del dfAll['date']

    #for minus in
    
    for plus in [['PartII_1','PartII_2'],['PartIII_1','PartIII_2','PartIII_3']]:
        tmpVar =plus[0]
        tmpValue = dfAll[plus[0]].values
        for var in plus[1:]:
            tmpVar = tmpVar +'_plus_' + var
            tmpValue = tmpValue + dfAll[var].values
        dfAll[tmpVar] = tmpValue
    
    dfAll['PartI_9'] = dfAll['PartI_5']-dfAll['PartI_6']-dfAll['PartI_7']
    dfAll['PartII_5'] = dfAll['PartII_2']-dfAll['PartII_3']-dfAll['PartII_4']
    
    for ratio in [['PartII_1','PartII_2'],['PartI_6','PartI_5'],['PartI_7','PartI_5'],['PartII_3','PartII_2'],['PartII_4','PartII_2'],['PartV_2','PartV_1'],['PartII_1','PartII_2']]:
        dfAll[ratio[0]+'_divided_'+ratio[1]] = dfAll[ratio[0]]/dfAll[ratio[1]]
        
    for multiply in [['PartI_1','PartI_5'],['PartI_2','PartI_5'],['PartI_3','PartI_5'],['PartI_4','PartI_5'],['PartVI_1','PartVI_2'],['PartV_2','PartV_5'],['PartV_2','PartV_7'],['PartV_1','PartV_9'],['PartV_1','PartV_10'],['PartV_1','PartV_11'],['PartV_1','PartV_12'],['PartV_1','PartV_13']]:
        dfAll[multiply[0]+'_multiply_'+multiply[1]] = dfAll[multiply[0]]*dfAll[multiply[1]]
    
    
    
    try:
        dfAll = dfAll.merge(dfAllPcentBySexAge,'inner','ID')
    except:
        print('No Pcent by sex and age')
        
    try:
        dfAll = dfAll.merge(dfrankMinus,'inner','ID')
    except:
        print('No Rank Minus')    

   
    
    
    ###填补缺失值
    if missing_step2:
        dfAll =dfAll.fillna(dfAll.median())
    
    dfTrain = dfAll.loc[dfAll['ID'].isin(train['ID'])]
    dfPred = dfAll.loc[dfAll['ID'].isin(pred['ID'])]
    
    return dfTrain,dfPred
    

In [4]:
dfTrain,dfPred = FeatAll(dfTrain,dfPred,True,True,True)
predictors = dfPred.columns.tolist()
predictors.remove('ID')
predictors.remove('Y')

C:\Users\Leo Mao\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
len(predictors)

2170

In [7]:
predictorsRF =[]
for i in range(150):
    t1 = datetime.datetime.now()
    predictorsRF += fs.RF_selection(dfTrain,predictors,int(random.random()*10000))[:1000]
    t2 = datetime.datetime.now()
    print("finish %d, use %d seconds"%(i,(t2-t1).seconds))
count = collections.Counter(predictorsRF)
dfpredictorsRF = pd.DataFrame(count,index=['count']).T
dfpredictorsRF.to_csv('../../Cache/Feat.csv',index=False)

finish 0, use 203 seconds
finish 1, use 204 seconds
finish 2, use 203 seconds
finish 3, use 203 seconds
finish 4, use 203 seconds
finish 5, use 203 seconds
finish 6, use 204 seconds
finish 7, use 203 seconds
finish 8, use 203 seconds
finish 9, use 203 seconds
finish 10, use 203 seconds
finish 11, use 203 seconds
finish 12, use 203 seconds
finish 13, use 203 seconds
finish 14, use 203 seconds
finish 15, use 203 seconds
finish 16, use 203 seconds
finish 17, use 203 seconds
finish 18, use 203 seconds
finish 19, use 210 seconds
finish 20, use 209 seconds
finish 21, use 205 seconds
finish 22, use 204 seconds
finish 23, use 204 seconds
finish 24, use 203 seconds
finish 25, use 203 seconds
finish 26, use 203 seconds
finish 27, use 203 seconds
finish 28, use 203 seconds
finish 29, use 204 seconds
finish 30, use 204 seconds
finish 31, use 203 seconds
finish 32, use 203 seconds
finish 33, use 204 seconds
finish 34, use 204 seconds
finish 35, use 203 seconds
finish 36, use 203 seconds
finish 37, 

In [8]:
dfpredictorsRF.to_csv('../../Cache/Feat.csv')

In [ ]:
testResult = dfTrain[['ID','Y']]
predResult = dfPred[['ID','Y']]

In [ ]:
n_splits=5
early_stop=50
params = {'max_depth':5, 'eta':0.01, 'silent':0,'objective':'binary:logistic','lambda':1,'subsample':0.8,'colsample_bytree':0.8,'eval_metric':'logloss'}
repeatRound =10
selectNum = 150
modelPredictors = predictors[:10]
for i in range(repeatRound):
    test_result,result,imp = mm.xgb_kfold(dfTrain,dfPred,tmpPredictor,n_splits,early_stop = early_stop,params=params,seed=int(random.random()*10000))
    for j in range(1,n_splits+1):
        imp['imp_fold%d'%j] = imp['imp_fold%d'%j]/imp['imp_fold%d'%j].sum()
    imp['sum_imp%d'%i] = imp[['imp_fold%d'%k for k in range(1,n_splits+1)]].sum(axis=1)
    if i ==0:
        importance = imp[['variable','sum_imp%d'%i]]
    else:
        importance = importance.merge(imp[['variable','sum_imp%d'%i]],'inner','variable')

importance['sum_total'] = importance[['sum_imp%d'%k for k in range(repeatRound)]].sum(axis=1)
tmpPredictor = importance.sort_values('sum_total',ascending=False)['variable'].values.tolist()[:selectNum] 
    

In [ ]:
###xgb
n_splits=5
early_stop=50
repeatRound =2
selectNum = 150
modelPredictors = tmpPredictor.copy()
for i in range(repeatRound):
    test_result,result,imp = mm.xgb_kfold(dfTrain,dfPred,modelPredictors,n_splits,early_stop = early_stop,params=params)
    result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
    if i ==0:
        tmpTest_result = test_result[['ID','score']]
        tmpResult = result[['ID','score']]
    else:
        tmpTest_result = tmpTest_result.merge(test_result[['ID','score']],'inner','ID')
        tmpResult = tmpResult.merge(result[['ID','score']],'inner','ID')
    tmpTest_result.rename(columns={'score':'score_%d'%i},inplace=True)
    tmpResult.rename(columns={'score':'score_%d'%i},inplace=True)
testResult['cateScoreXGB'] = tmpTest_result[['score_%d'%i for i in range(repeatRound)]].mean(axis=1).values
predResult['cateScoreXGB'] = tmpResult[['score_%d'%i for i in range(repeatRound)]].mean(axis=1).values

In [ ]:
###RF
n_splits=5
early_stop=50
repeatRound =2
selectNum = 150
modelPredictors = predictors[:10]
for i in range(repeatRound):
    test_result,result,imp = mm.rf_kfold(dfTrain,dfPred,modelPredictors,n_splits,seed=int(random.random()*10000))
    result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
    if i ==0:
        tmpTest_result = test_result[['ID','score']]
        tmpResult = result[['ID','score']]
    else:
        tmpTest_result = tmpTest_result.merge(test_result[['ID','score']],'inner','ID')
        tmpResult = tmpResult.merge(result[['ID','score']],'inner','ID')
    tmpTest_result.rename(columns={'score':'score_%d'%i},inplace=True)
    tmpResult.rename(columns={'score':'score_%d'%i},inplace=True)
testResult['cateScoreRF'] = tmpTest_result[['score_%d'%i for i in range(repeatRound)]].mean(axis=1).values
predResult['cateScoreRF'] = tmpResult[['score_%d'%i for i in range(repeatRound)]].mean(axis=1).values

In [ ]:
###catboost
n_splits=5
early_stop=50
repeatRound =2
selectNum = 150
modelPredictors = predictors[:10]
for i in range(repeatRound):
    test_result,result,imp = mm.catboost_kfold(dfTrain,dfPred,modelPredictors,n_splits,early_stop=50,seed=int(random.random()*10000))
    result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
    if i ==0:
        tmpTest_result = test_result[['ID','score']]
        tmpResult = result[['ID','score']]
    else:
        tmpTest_result = tmpTest_result.merge(test_result[['ID','score']],'inner','ID')
        tmpResult = tmpResult.merge(result[['ID','score']],'inner','ID')
    tmpTest_result.rename(columns={'score':'score_%d'%i},inplace=True)
    tmpResult.rename(columns={'score':'score_%d'%i},inplace=True)
testResult['cateScoreCat'] = tmpTest_result[['score_%d'%i for i in range(repeatRound)]].mean(axis=1).values
predResult['cateScoreCat'] = tmpResult[['score_%d'%i for i in range(repeatRound)]].mean(axis=1).values

## 分类模型

In [ ]:
selectNum = 150
for i in range(1,n_splits+1):
    imp['imp_fold%d'%i] = imp['imp_fold%d'%i]/imp['imp_fold%d'%i].sum()
imp['sum_imp'] = imp[['imp_fold%d'%i for i in range(1,n_splits+1)]].sum(axis=1)

tmpPredictor = imp.sort_values('sum_imp',ascending=False)['variable'].values.tolist()[:selectNum]
imp.sort_values('sum_imp',ascending=False)

In [ ]:
test_result,result,imp_selected = mm.xgb_kfold(dfTrain,dfPred,tmpPredictor,n_splits,early_stop = early_stop,params=params)

In [ ]:
mt.evaluate_performance(test_result['target'],test_result['score'])
imp.sort_values('sum_imp',ascending=False)

## 回归模型

In [ ]:
##将分类打分放入原始数据
dfTrain['scoreBinary'] = test_result['score']
result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
dfPred['scoreBinary'] = result['score'].tolist()

predictorReg = predictors.copy()
predictorReg.append('scoreBinary')

dfTrain['Y'] = saveY

##将分数以rank形式保存一份
'''dfTrain['scoreBinaryPcent'] = dfTrain['scoreBinary'].rank()/float(dfTrain.shape[0])
dfPred['scoreBinaryPcent'] = dfPred['scoreBinary'].rank()/float(dfPred.shape[0])
predictorReg.append('scoreBinaryPcent')'''

In [ ]:
print(dfTrain['scoreBinary'].mean())
print(dfTrain['scoreBinary'].max())
print(dfTrain['scoreBinary'].min())
print('--------------')
print(dfPred['scoreBinary'].mean())
print(dfPred['scoreBinary'].max())
print(dfPred['scoreBinary'].min())

In [ ]:
predictorsRF = fs.RF_selection(dfTrain,predictorReg)
modelPredictors = predictorsRF[:500]

In [ ]:
n_splits=5
early_stop=50
params = {'max_depth':5, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1,'subsample':0.8,'colsample_bytree':0.8}
test_result,result,imp = mm.xgb_kfold(dfTrain,dfPred,modelPredictors,n_splits,early_stop=early_stop,params=params)

In [ ]:
selectNum = 100
for i in range(1,n_splits+1):
    imp['imp_fold%d'%i] = imp['imp_fold%d'%i]/imp['imp_fold%d'%i].sum()
imp['sum_imp'] = imp[['imp_fold%d'%i for i in range(1,n_splits+1)]].sum(axis=1)

tmpPredictor = imp.sort_values('sum_imp',ascending=False)['variable'].values.tolist()[0:selectNum+1]
imp.sort_values('sum_imp',ascending=False)

In [ ]:
test_result,result,imp_selected = mm.xgb_kfold(dfTrain,dfPred,tmpPredictor,n_splits,early_stop=early_stop,params=params)

In [ ]:
for i in range(1,n_splits+1):
    imp_selected['imp_fold%d'%i] = imp_selected['imp_fold%d'%i]/imp_selected['imp_fold%d'%i].sum()
imp_selected['sum_imp'] = imp_selected[['imp_fold%d'%i for i in range(1,n_splits+1)]].sum(axis=1)
imp_selected.sort_values('sum_imp',ascending=False).reset_index()

In [ ]:
result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
print((result['score']>6.8).sum())
print(result['score'].min())
print(result['score'].max())
print(result['score'].mean())
print(result['score'].median())
print("-------------------------")
print(result.iloc[938,:])

In [ ]:
n_splits=5
other_note ='_classfier_first'
result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
result.to_csv('../../Submission/result/result_%s'%today+other_note+'.csv',index=False)
submit['score'].to_csv('../../Submission/submit_%s'%today+other_note+'.csv',header=False,index=False)
test_result.to_csv('../../Submission/test/test_result_%s'%today+other_note+'.csv',index=False)
imp.to_csv('../../Submission/imp/importance_%s'%today+other_note+'.csv',index=False)